In [32]:
import datetime
import pandas as pd
import numpy as np

# Read the data
#data = pd.read_csv('C:\\temp\\sizing\\data.csv')
data = pd.read_csv('data.csv')
data['Date'] = pd.to_datetime(data['Date'])
data['StartDate'] = pd.to_datetime(data['StartDate'])
data['EndDate'] = pd.to_datetime(data['EndDate'])

# Tickets:     clientes reais
# TicketsPrev: clientes estimados
# Persons:     número de pessoas em caixa (estimado)

# RosterCode, Date, StartDate, EndDate, Tickets, TicketsPrev, Persons

#data.head(25)
data

,RosterCode,Date,StartDate,EndDate,Tickets,TicketsPrev,Persons
0,10,2021-11-01,2021-11-01 09:00:00,2021-11-01 10:00:00,32,55,2
1,10,2021-11-01,2021-11-01 10:00:00,2021-11-01 11:00:00,57,69,3
2,10,2021-11-01,2021-11-01 11:00:00,2021-11-01 12:00:00,87,112,4
3,10,2021-11-01,2021-11-01 12:00:00,2021-11-01 13:00:00,97,118,4
4,10,2021-11-01,2021-11-01 13:00:00,2021-11-01 14:00:00,100,130,4
...,...,...,...,...,...,...,...
201614,72,2022-10-31,2022-10-31 17:00:00,2022-10-31 18:00:00,54,59,2
201615,72,2022-10-31,2022-10-31 18:00:00,2022-10-31 19:00:00,86,90,3
201616,72,2022-10-31,2022-10-31 19:00:00,2022-10-31 20:00:00,114,108,4
201617,72,2022-10-31,2022-10-31 20:00:00,2022-10-31 21:00:00,103,81,3


In [34]:
data_filtered = data.drop(columns=['Date', 'EndDate' , 'Tickets'])
data_filtered

,RosterCode,StartDate,TicketsPrev,Persons
0,10,2021-11-01 09:00:00,55,2
1,10,2021-11-01 10:00:00,69,3
2,10,2021-11-01 11:00:00,112,4
3,10,2021-11-01 12:00:00,118,4
4,10,2021-11-01 13:00:00,130,4
...,...,...,...,...
201614,72,2022-10-31 17:00:00,59,2
201615,72,2022-10-31 18:00:00,90,3
201616,72,2022-10-31 19:00:00,108,4
201617,72,2022-10-31 20:00:00,81,3


In [35]:
#stores = pd.value_counts(data_filtered['RosterCode'].values.ravel()).sort_values()
stores = pd.value_counts(data_filtered['RosterCode'].values.ravel())
stores_ids = list(stores.index)
stores_ids.sort()

#print(stores)
#print(stores_ids)
#print(stores.array)

print('---------------------------------------------------------------------')
print('Nº stores: ' + str(len(stores)))
print('---------------------------------------------------------------------')

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

def model_metrics(y_true, y_pred):
    d_metrics={}
    d_metrics['rmse'] = np.sqrt(mean_squared_error(y_true, y_pred))
    d_metrics['mae'] = mean_absolute_error(y_true, y_pred)
    d_metrics['r2_score'] = r2_score(y_true, y_pred)
    
    return d_metrics

d = {'Store ID': [], 'N Records': [], 'N Records Agg': [], 'N Records Train': [], 'N Records Test': [], 'RMSE': [], 'MAE': [], 'R2_SCORE': []}
r = pd.DataFrame(data=d)

for id in stores_ids:
    #print('Store Id: ' + str(id))
    store_data = data_filtered.loc[data['RosterCode'] == id]
    store_data = store_data.drop(columns=['RosterCode'])
    
    n_store_rec = store_data.shape[0]
    #print('Nº records: ' + str(n_store_rec))
    
    store_data_filtered = pd.DataFrame(store_data.resample('H', on='StartDate').sum())
    
    #print(store_data_filtered)
    n_store_rec_agg = store_data_filtered.shape[0]
    #print('Nº records agg: ' + str(n_store_rec_agg))
    
    store_data_filtered.reset_index(inplace=True)
    #print(store_data_filtered)
    
    store_data_filtered['Month'] = store_data_filtered['StartDate'].apply(lambda x: x.month)
    store_data_filtered['Day'] = store_data_filtered['StartDate'].apply(lambda x: x.weekday())
    store_data_filtered['Hour'] = store_data_filtered['StartDate'].apply(lambda x: x.hour)
    #print(store_data_filtered)
    
    store_data_filtered = store_data_filtered.drop(columns=['StartDate'])
    #print(store_data_filtered)
    
    store_data_filtered = store_data_filtered[['Month', 'Day', 'Hour', 'TicketsPrev', 'Persons']]
    #print(store_data_filtered)
    
    N = n_store_rec_agg
    K = N * 80 / 100
    #print(N, K, int(K))
    K = int(K)
    
    filename = 'loja_' + str(id) + '_data.csv'
    store_data_filtered.to_csv(filename, index=False)

    X = store_data_filtered.iloc[:,:-1].values
    y = store_data_filtered.iloc[:,-1].values

    #print(X)
    #print(y)
    
    X_train = X[:-K,:]
    y_train = y[:-K]
    X_test = X[-K:]
    y_test = y[-K:]
    
    regr = RandomForestRegressor()
    regr.fit(X_train, y_train)
    
    y_pred = regr.predict(X_test)
    metrics = model_metrics(y_test, y_pred)
    #print(model_metrics(y_test, y_pred))
    
    r = pd.concat([r, pd.DataFrame([[id, n_store_rec, n_store_rec_agg, K, N-K, metrics['rmse'], metrics['mae'], metrics['r2_score']]], columns=r.columns)], ignore_index=True)

    #break

#print(r)
filename = 'results_data.csv'
r.to_csv(filename, index=False)

---------------------------------------------------------------------
Nº stores: 49
---------------------------------------------------------------------
